In [ ]:
path = None

In [21]:
import re
import tempfile

from IPython.display import display
import pandas as pd
from PIL import Image

from flytekit.types.file import FlyteFile

# Metrics Report

In [ ]:
df = pd.read_parquet(path)
df["correct"] = ((df.prediction > 0.5) == df.label).astype(int)
df.sort_values(by="correct")[["label", "prediction", "description"]]

In [ ]:
acc = df["correct"].sum() / len(df)
print(f"accuracy = {acc}")

In [26]:
def download_file_from_s3(s3_path):
    # Extract the bucket and key from the S3 path
    match = re.match(r's3://([^/]+)/(.+)', s3_path)
    if not match:
        raise ValueError(f"Invalid S3 path: {s3_path}")

    bucket, key = match.groups()

    # Create a boto3 client for S3
    s3_client = boto3.client('s3')

    # Download the file
    tmp = tempfile.mktemp()
    s3_client.download_file(bucket, key, tmp)
    return tmp

# False Positives 

In [ ]:
for i, img in df.iterrows():
    if (img.prediction > 0.5) == 1 and img.label == 0.0:
        print(f"Image {i}: {img.description}")
        display(Image.open(download_file_from_s3(img.remote_source)))

# True Positives & True Negatives

In [ ]:
for i, img in df.iterrows():
    if (img.prediction > 0.5) == img.label:
        print(f"Image {i}: {img.description}")
        display(Image.open(download_file_from_s3(img.remote_source)))